Importing the Dependencies

In [6]:
pip install gtts

In [7]:
pip install SpeechRecognition

In [8]:
pip install googletrans==3.1.0a

In [9]:
pip install playsound

In [10]:
from gtts import gTTS
import speech_recognition as sr
from googletrans import Translator
import playsound
import os

In [11]:
translator = Translator()

import speech_recognition as sr

# Create an instance of the Recognizer class
recognizer = sr.Recognizer()

# Open an audio file using sr.AudioFile()
audio_file = sr.AudioFile("Recording.wav")

# Use the recognizer to recognize speech from the audio file
with audio_file as source:
    audio_data = recognizer.record(source)  # Record the audio data from the file
    text = recognizer.recognize_google(audio_data, language = 'bn')  # Recognize the speech using Google's speech recognition API

print("Recognized text:", text)

Recognized text: এই বছর পেস্টিসাইড এর অভাবে আমার চাষের ভীষণ ক্ষতি হয়েছে পরের বছর আরো ভালো পেস্টিসাইড এর ব্যবস্থা করা হোক


In [12]:
translator = Translator()
translated_text = translator.translate(text, src='bn', dest='en')

print("Translated text (to English):", translated_text.text)

Translated text (to English): Due to lack of pesticide this year, my crop has suffered a lot. Next year, better pesticide should be arranged


In [13]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [16]:
# loading the dataset to a pandas DataFrame
dataset = pd.read_csv('/content/final (3).csv')

In [17]:
print(dataset.shape)
dataset.columns

(18898, 2)


Index(['Department ', 'Complain '], dtype='object')

In [18]:
# print the first 5 rows of the dataframe
#dataset['c']
dataset.head()

,Department,Complain
0,0,"""There is no hospital or clinic within a 20-ki..."
1,0,"""The government should invest in telemedicine ..."
2,0,"""The government should prioritize healthcare f..."
3,0,"""Government healthcare centers lack facilities..."
4,0,"""The government should create a unified health..."


In [19]:
# counting the number of missing values in the dataset
dataset.isnull().sum()

Department     0
Complain       0
dtype: int64

In [20]:
# separating the data & label
X = dataset.drop(columns='Department ', axis=1)
Y = dataset['Department ']

In [21]:
print(X)
print(Y)

                                               Complain 
0      "There is no hospital or clinic within a 20-ki...
1      "The government should invest in telemedicine ...
2      "The government should prioritize healthcare f...
3      "Government healthcare centers lack facilities...
4      "The government should create a unified health...
...                                                  ...
18893  How can I file a complaint related to workplac...
18894  Can you explain the process for renouncing Ind...
18895  How can I apply for Indian citizenship through...
18896  What is the procedure for resolving disputes r...
18897  What rights and responsibilities do foreign na...

[18898 rows x 1 columns]
0        0
1        0
2        0
3        0
4        0
        ..
18893    3
18894    3
18895    3
18896    3
18897    3
Name: Department , Length: 18898, dtype: int64


Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [24]:
dataset['Complain '] = dataset['Complain '].apply(stemming)

In [25]:
#separating the data and label
X = dataset['Complain '].values
Y = dataset['Department '].values

In [26]:
print(X)

['hospit clinic within kilomet radiu villag make extrem difficult access healthcar emerg'
 'govern invest telemedicin infrastructur improv access specialist'
 'govern priorit healthcar homeless margin popul' ...
 'appli indian citizenship natur'
 'procedur resolv disput relat intellectu properti infring'
 'right respons foreign nation india']


In [27]:
print(type(X))

<class 'numpy.ndarray'>


In [28]:
print(Y)

[0 0 0 ... 3 3 3]


In [29]:
Y.shape

(18898,)

In [30]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [31]:
print(X)

  (0, 1358)	0.2413773003363203
  (0, 1321)	0.3192927204092433
  (0, 989)	0.46194143551558364
  (0, 728)	0.18518013381201417
  (0, 671)	0.46194143551558364
  (0, 575)	0.2079213291469036
  (0, 548)	0.14969777677303836
  (0, 454)	0.29411656387709556
  (0, 378)	0.2417355290413319
  (0, 309)	0.23536926831165206
  (0, 180)	0.31459837313747735
  (0, 5)	0.11732665701325262
  (1, 1214)	0.5452053589987971
  (1, 1142)	0.4457574018342694
  (1, 654)	0.37694490469999825
  (1, 627)	0.3857702224755298
  (1, 597)	0.3524381870098648
  (1, 521)	0.2144944161175508
  (1, 5)	0.207183009178925
  (2, 942)	0.4157820332180444
  (2, 920)	0.45597294839109576
  (2, 736)	0.45458700841335453
  (2, 571)	0.5311529005622533
  (2, 548)	0.2804551758380928
  (2, 521)	0.2275656246895086
  :	:
  (18893, 539)	0.40892506550778107
  (18893, 477)	0.3538361169481323
  (18893, 328)	0.39529155796810805
  (18893, 196)	0.37931621249479086
  (18894, 1034)	0.5535221283605066
  (18894, 950)	0.28824194705433903
  (18894, 614)	0.47208976

Splitting the dataset to training & test data

In [32]:
X.shape

(18898, 1375)

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [34]:
model_tree=DecisionTreeClassifier(random_state=42)
model_tree.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=42)

In [35]:
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train, Y_train)

SVC(kernel='linear', random_state=42)

In [36]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [37]:
model_forest=RandomForestClassifier(n_estimators=1000, random_state=42)
model_forest.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

Evaluation

accuracy score

In [38]:
# accuracy score on the training data
X_train_prediction = model_forest.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  1.0


In [39]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [40]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  1.0


In [41]:
X_train.shape

(15118, 1375)

In [42]:
# accuracy score on the test data
X_test_prediction = model_forest.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9994708994708995


In [43]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [44]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  1.0


Making a Predictive System

In [45]:
print(model.predict(vectorizer.transform(pd.DataFrame([translated_text.text], columns = ['comp'])['comp'].values)))

[2]


In [ ]:
print(model_forest.predict(vectorizer.transform(pd.DataFrame(["I'm deeply concerned about the rampant use of substandard and counterfeit agricultural inputs in our region. The availability of fake pesticides, fertilizers, and seeds not only jeopardizes our crop yields but also poses health risks to farmers and consumers alike. I urge the Agriculture Department to strengthen its monitoring and enforcement efforts to curb the sale of counterfeit agricultural products and ensure that farmers have access to genuine and safe inputs for sustainable farming practices."], columns = ['comp'])['comp'].values)))

[2]


In [ ]:
b="I'm deeply concerned about the rampant use of substandard and counterfeit agricultural inputs in our region. The availability of fake pesticides, fertilizers, and seeds not only jeopardizes our crop yields but also poses health risks to farmers and consumers alike. I urge the Agriculture Department to strengthen its monitoring and enforcement efforts to curb the sale of counterfeit agricultural products and ensure that farmers have access to genuine and safe inputs for sustainable farming practices."

In [ ]:
a="i play football at state level and recently got injured, there is no government subsidy for sports athletes representating states of country, kindly include some monetary benifit for such athletes with difficult financial background."

In [ ]:
print(model.predict(vectorizer.transform(pd.DataFrame([stemming(a)], columns = ['comp'])['comp'].values)))

[2]


In [ ]:
print(model_forest.predict(vectorizer.transform(pd.DataFrame([stemming(a)], columns = ['comp'])['comp'].values)))

[3]


In [ ]:
print(model_tree.predict(vectorizer.transform(pd.DataFrame([stemming(b)], columns = ['comp'])['comp'].values)))

[2]


In [ ]:
print(svm_classifier.predict(vectorizer.transform(pd.DataFrame([stemming(b)], columns = ['comp'])['comp'].values)))

[2]


In [98]:
d = {
    0: 'Health',
    1: 'Education',
    2: 'Agriculture',
    3: 'Justice'
}

In [ ]:
import nltk
import random
import re
from nltk.chat.util import Chat, reflections

# Import NLTK's chat data for greetings and farewells
nltk.download("nltk.chat")

# Define custom patterns and responses for your chatbot
patterns = [
    (r"(hi|hello|hey)( me)?", ["Hello!", "Hi there!", "How can I assist you today?"]),
    (r"(help|assist|support)( me)?", ["Of course! I'm here to assist you. Please let me know your question or concern."]),
    (r"(capabilities|what can you do)( me)?", ["I can help you classify complaints into government departments. Just describe the issue."]),
    (r"(complaint)( me)?", ["I'm here to help with complaints. Please describe your issue."]),
    (r"(agriculture|education|health|transport|finance)( me)?", ["This issue seems related to the {} department."]),
    (r"(thank you|thanks|bye|goodbye)( me)?", ["You're welcome! Have a great day!"]),
    (r".*", ["I'm here to assist you. Please provide more details."]),  # Default response
]

# Create a chatbot using NLTK's Chat class
chatbot = Chat(patterns, reflections)

# Define a function to interact with the chatbot
def chat_with_bot():
    print("Chatbot: Hello! I'm here to help with government department complaints.")
    print("Chatbot: You can type 'bye' to exit the conversation.")
    while True:
        user_input = input("You: ")

        if user_input.lower() == "bye":
            print("Chatbot: Goodbye!")
            break

        # Check if any pattern keyword is present in the user input
        for pattern, responses in patterns:
            match = re.search(pattern, user_input, re.IGNORECASE)
            if match:
                if 'complaint' in pattern:
                  response = random.choice(responses).format(*match.groups())
                  print("Chatbot:", response)
                  user_input2 = input("You: ")

                  category = model_forest.predict(vectorizer.transform(pd.DataFrame([stemming(user_input2)], columns = ['comp'])['comp'].values))
                  dept = d[int(category)]
                  response = f"This issue seems related to the {dept} department."
                  print("Chatbot: ", response)
                  break

                else:

                  # If the pattern contains capturing groups, use them in the response
                  if "(" in pattern and ")" in pattern:
                      response = random.choice(responses).format(*match.groups())
                  else:
                      response = random.choice(responses)
                  print("Chatbot:", response)
                  break

        else:
            print("Chatbot: I couldn't understand your request. Can you please rephrase it?")

    # return user_input2, dept

# Start the chat
chat_with_bot()


[nltk_data] Error loading nltk.chat: Package 'nltk.chat' not found in
[nltk_data]     index


Chatbot: Hello! I'm here to help with government department complaints.
Chatbot: You can type 'bye' to exit the conversation.
You: heyy
Chatbot: How can I assist you today?
You: i need some help
Chatbot: Of course! I'm here to assist you. Please let me know your question or concern.
You: I need crops for my field
Chatbot: I'm here to assist you. Please provide more details.
You: I have a complaint
Chatbot: I'm here to help with complaints. Please describe your issue.
You: I need crops for my field
Chatbot:  This issue seems related to the Agriculture department.
You: thanks
Chatbot: You're welcome! Have a great day!
You: bye
Chatbot: Goodbye!
